In [1]:
import numpy as np
from tqdm.auto import tqdm
from load import load_data
from convert import convert
from pmc2 import *

In [2]:
data = load_data(7)
d = convert(data)

In [3]:
print(d['C'][:,3])
print(d['E'][:,3])
print(d['o'][:,3])

[ 33.3  74.8 106.1 130.4 153.3 172.9 211.8]
[]
[138.6 146.1 158.3 177.3 203.2 231.6 240.4 165.4 192.7 215.8 225.3 239.4
 249.8 195.6 212.7 240.9 264.5 274.7 297.5 229.  254.4 284.4 325.8 332.5]


In [4]:
lin_C = d['pC'].size
lin_E = d['pE'].size
lin_o = d['po'].size

In [5]:
## INITIALIZATION
data_C = get_C(data)[np.argsort(get_C(data)[:, 3])] # sorted with p (mean stress)
data_E = get_E(data)[np.argsort(get_E(data)[:, 3])] # sorted with p (mean stress)
data_o = get_o(data)[np.argsort(get_o(data)[:, 5])] # sorted with teta (Lodge angle)
#print(data_C)

if lin_C ==0 :
    P1_C=data_C
    P2_C=data_C
    nb_comb = lin_E*lin_o;
    
if lin_E ==0 :
    P1_E=data_E
    P2_E=data_E
    nb_comb = lin_C*lin_o;

if lin_o ==0 :
    P1_o=data_o
    P2_o=data_o
    nb_comb = lin_E*lin_C;
    
if get_C(data).size !=0 and get_E(data).size !=0 and get_o(data).size !=0:
    nb_comb = lin_E*lin_C*lin_o;
    
mean_error = []
a=0
#print(lin_C)
#print(P2_C)
#print(P1_C)
#print(nb_comb)
#print(range(lin_C-1,0,-1))
#for i in range(lin_C-1,0,-1):
#    print(i)
#print(data_E[:i+1,:])
#print(data_E)

In [6]:
## ITERATIONS
if get_C(data).size ==0 :
    bp_set = np.zeros((nb_comb//1,2))
    for j in tqdm(range(lin_E-1,0,-1)) :
        for k in range(lin_o-1,0,-1) :
            bp_set[a,0] = j
            bp_set[a,1] = k
            P1_E=data_E[j:lin_E,:]
            P2_E=data_E[:j,:]
            P1_o=data_o[k:lin_o,:]
            P2_o=data_o[:k,:]
            P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o) 
            P1_coeff = [P1.A,P1.B,P1.C]
            P2_coeff = [P2.A,P2.B,P2.C]
            #print(P1.sol)
            sP1 = np.dot(np.transpose(P1.sig123),P1_coeff)
            sP2 = np.dot(np.transpose(P2.sig123),P2_coeff)
            errP1 = abs(1-sP1)
            errP2 = abs(1-sP2)
            sumP1 = np.sum(errP1)
            sumP2 = np.sum(errP2)
            mean_error.append((sumP1 + sumP2)/(len(errP1)+len(errP2)))
            a=a+1

    mean_error = np.array(mean_error)
    print(mean_error)
    min_error_index = np.nanargmin(mean_error)
    print(min_error_index)
    bp1, bp2=bp_set[min_error_index]
    print(bp1,bp2)
    P1_E=data_E[int(bp1):lin_E,:]
    P2_E=data_E[:int(bp1),:]
    P1_o=data_o[int(bp2):lin_o,:]
    P2_o=data_o[:int(bp2),:]
    
elif get_E(data).size ==0 :
    bp_set = np.zeros((nb_comb//1,2))
    for i in tqdm(range(lin_C-1,0,-1)) :
        for k in range(lin_o-1,0,-1) :
            bp_set[a,0] = i
            bp_set[a,1] = k
            P1_C=data_C[i:lin_C,:]
            P2_C=data_C[:i,:]
            P1_o=data_o[k:lin_o,:]
            P2_o=data_o[:k,:]
            P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o) 
            P1_coeff = [P1.A,P1.B,P1.C]
            P2_coeff = [P2.A,P2.B,P2.C]
            #print(P1.sol)
            sP1 = np.dot(np.transpose(P1.sig123),P1_coeff)
            sP2 = np.dot(np.transpose(P2.sig123),P2_coeff)
            errP1 = abs(1-sP1)
            errP2 = abs(1-sP2)
            sumP1 = np.sum(errP1)
            sumP2 = np.sum(errP2)
            mean_error.append((sumP1 + sumP2)/(len(errP1)+len(errP2)))
            a=a+1
            
    mean_error = np.array(mean_error)
    print(mean_error)
    min_error_index = np.nanargmin(mean_error)
    print(min_error_index)
    bp1, bp2=bp_set[min_error_index]
    print(bp1,bp2)
    P1_C=data_C[int(bp1):lin_C,:]
    P2_C=data_C[:int(bp1),:]
    P1_o=data_o[int(bp2):lin_o,:]
    P2_o=data_o[:int(bp2),:]
    
elif get_o(data).size ==0 :
    bp_set = np.zeros((nb_comb//1,2))
    for i in tqdm(range(lin_C-1,0,-1)) :
        for j in range(lin_E-1,0,-1) :
            bp_set[a,0] = i
            bp_set[a,1] = j
            P1_C=data_C[i:lin_C,:]
            P2_C=data_C[:i,:]
            P1_E=data_E[j:lin_E,:]
            P2_E=data_E[:j,:]
            P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o) 
            P1_coeff = [P1.A,P1.B,P1.C]
            P2_coeff = [P2.A,P2.B,P2.C]
            #print(P1.sol)
            sP1 = np.dot(np.transpose(P1.sig123),P1_coeff)
            sP2 = np.dot(np.transpose(P2.sig123),P2_coeff)
            errP1 = abs(1-sP1)
            errP2 = abs(1-sP2)
            sumP1 = np.sum(errP1)
            sumP2 = np.sum(errP2)
            mean_error.append((sumP1 + sumP2)/(len(errP1)+len(errP2)))
            a=a+1

    mean_error = np.array(mean_error)
    print(mean_error)
    min_error_index = np.nanargmin(mean_error)
    print(min_error_index)
    bp1,bp2 = bp_set[min_error_index]
    print(bp1,bp2)
    P1_C=data_C[int(bp1):lin_C,:]
    P2_C=data_C[:int(bp1),:]
    P1_E=data_E[int(bp2):lin_E,:]
    P2_E=data_E[:int(bp2),:]
    
elif get_C(data).size !=0 and get_E(data).size !=0 and get_o(data).size !=0:
    bp_set = np.zeros((nb_comb//1,3))
    for i in tqdm(range(lin_C-1,0,-1)) :
        for j in range(lin_E-1,0,-1) :
            for k in range(lin_o-1,0,-1) :
                bp_set[a,0] = i
                bp_set[a,1] = j
                bp_set[a,2] = k
                P1_C=data_C[i:lin_C,:]
                P2_C=data_C[:i,:]
                P1_E=data_E[j:lin_E,:]
                P2_E=data_E[:j,:]
                P1_o=data_o[k:lin_o,:]
                P2_o=data_o[:k,:]
                P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o) 
                P1_coeff = [P1.A,P1.B,P1.C]
                P2_coeff = [P2.A,P2.B,P2.C]
                #print(P1.sol)
                sP1 = np.dot(np.transpose(P1.sig123),P1_coeff)
                sP2 = np.dot(np.transpose(P2.sig123),P2_coeff)
                errP1 = abs(1-sP1)
                errP2 = abs(1-sP2)
                sumP1 = np.sum(errP1)
                sumP2 = np.sum(errP2)
                mean_error.append((sumP1 + sumP2)/(len(errP1)+len(errP2)))
                a=a+1
        
    mean_error = np.array(mean_error)
    print(mean_error)
    min_error_index = np.nanargmin(mean_error)
    print(min_error_index)
    i, j, k = bp_set[min_error_index]
    print(i,j,k)
    P1_C=data_C[int(i):lin_C,:]
    P2_C=data_C[:int(i),:]
    P1_E=data_E[int(j):lin_E,:]
    P2_E=data_E[:int(j),:]
    P1_o=data_o[int(k):lin_o,:]
    P2_o=data_o[:int(k),:]
    

print(np.nanmin(mean_error))
P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o)
print(P2_C)
print(P1_C)
#print(P2_E)
#print(P1_E)
print(P2_o)
print(P1_o)
print(P1.sol)
print(P2.sol)


[0.45527065 0.09531965 0.09216971 0.09671077 0.09222757 0.08416874
 0.0845059  0.08641933 0.08141113 0.0824318  0.08386158 0.07895199
 0.07742155 0.07615329 0.07544693 0.07405482 0.07226208 0.07128829
 0.06872998 0.06934424 0.06877258 0.07021753        nan 0.09154126
 0.09350566 0.08976223 0.09252774 0.08753565 0.07958435 0.07993984
 0.08261647 0.07881741 0.08126977 0.08416994 0.08114926 0.0805062
 0.08165259 0.08087951 0.07942874 0.07867605 0.07971626 0.07595851
 0.0757111  0.07396441 0.0735183         nan 0.0900366  0.09349911
 0.08966527 0.09144217 0.08764813 0.08029505 0.08074634 0.08296614
 0.07913009 0.0804845  0.08326264 0.0806681  0.08121975 0.0839504
 0.08375506 0.08274585 0.0830752  0.08527247 0.07999196 0.07990258
 0.0776615  0.07619069 0.0648378  0.08907099 0.0933048  0.08975024
 0.09076989 0.08833003 0.08101078 0.08117197 0.08278947 0.07854781
 0.07966835 0.08242929 0.07972057 0.08052627 0.08370876 0.08467315
 0.08517489 0.08555017        nan 0.0843441  0.08403486 0.08122

/Users/heloisefuselier/Dropbox/PDM/Travail/PDM helo/Python/pmc2.py:195: RuntimeWarning: invalid value encountered in arcsin
  self.phyE = arcsin(3*be/(6*self.Vo-be))
/Users/heloisefuselier/Dropbox/PDM/Travail/PDM helo/Python/pmc2.py:193: RuntimeWarning: invalid value encountered in arcsin
  self.phyC = arcsin(3*bc/(6*self.Vo+bc))


In [7]:
data_1 = data[np.argsort(data[:, 4])]